In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)
print(keras.__version__)

2.5.0
2.5.0


In [2]:
n_outputs = 3

In [3]:
import numpy as np
import os
from os import listdir 
from os.path import isfile, join
import cv2

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split

import random
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [4]:
# Searching training data

label_folder = []
total_size = 0
data_path = r"C:\Users\ESA LAB\Desktop\CNN_FOG_711083104\CNN_FOG - 1 - 50%OL-2-5_preFog\dataset\RP"

#os.walk() generates the file names(dirpath, dirnames, filenames) 
#in a directory tree by walking the tree either top-down or bottom-up.
for root, dirts, files in os.walk(data_path): 
    for dirt in dirts:
        label_folder.append(dirt)
    total_size += len(files)

    
print("found",total_size,"files.")
print("folder:",label_folder)

found 24884 files.
folder: ['0', '1', '2']


In [5]:
#Load training images

base_x_train = []
base_y_train = []

for i in range(len(label_folder)):
    labelPath = data_path+r'\\'+label_folder[i]
    
    #listdir() returns a list containing the names of the entries in the directory given by path.
    #isfile() is used to check whether the specified path is an existing regular file or not.
    FileName = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]
    
    for j in range(len(FileName)):
        path = labelPath+r'\\'+FileName[j]
        
        #use cv2.imread read image.
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        
        base_x_train.append(img)
        base_y_train.append(int(label_folder[i]))


print(np.array(base_x_train).shape)
print(np.array(base_y_train).shape)

(24884, 72, 72, 3)
(24884,)


In [6]:
# normalization the data, every value is between 0 and 1

base_x_train = np.array(base_x_train) 
base_y_train = np.array(base_y_train)


base_x_train = base_x_train.astype('float')/255.0

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X, x_test, y, y_test = train_test_split(base_x_train, base_y_train, test_size=0.2, random_state=42)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(72, 72, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(128, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), padding = "same", activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(n_outputs, activation = "softmax")
])
	

In [10]:
# using Stratified KFold cross-validation to split datas
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
skf.get_n_splits(base_x_train, base_y_train)

10

In [12]:
epoch = 30
batch_size = 100


for i, (train, test) in enumerate(skf.split(X, y)):
    print("==> Fold #%d" % i)

    x_train, x_val = X[train], X[test]
    y_train, y_val = y[train], y[test]

    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)

    model.compile(
        loss="categorical_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"]
    )
    history = model.fit(
        x_train, 
        y_train, 
        epochs=epoch, 
        batch_size=batch_size,
        validation_data=(x_val, y_val)
    )

==> Fold #0
Epoch 1/30
180/180 [==============================] - 9s 27ms/step - loss: 1.0819 - accuracy: 0.6059 - val_loss: 0.6045 - val_accuracy: 0.8388
Epoch 2/30
180/180 [==============================] - 4s 23ms/step - loss: 0.5178 - accuracy: 0.8168 - val_loss: 0.4529 - val_accuracy: 0.8393
Epoch 3/30
180/180 [==============================] - 4s 23ms/step - loss: 0.4507 - accuracy: 0.8342 - val_loss: 0.4737 - val_accuracy: 0.8122
Epoch 4/30
180/180 [==============================] - 4s 23ms/step - loss: 0.4268 - accuracy: 0.8375 - val_loss: 0.7952 - val_accuracy: 0.5766
Epoch 5/30
180/180 [==============================] - 4s 23ms/step - loss: 0.4261 - accuracy: 0.8390 - val_loss: 0.4678 - val_accuracy: 0.8368
Epoch 6/30
180/180 [==============================] - 4s 23ms/step - loss: 0.4115 - accuracy: 0.8407 - val_loss: 0.5582 - val_accuracy: 0.8388
Epoch 7/30
180/180 [==============================] - 4s 23ms/step - loss: 0.4030 - accuracy: 0.8425 - val_loss: 0.3917 - val_accu

In [13]:
model.save("my_model.hdf5")

In [14]:
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score

In [19]:
y_test = to_categorical(y_test)
y_pred = model.predict(x_test)

y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)

confus = confusion_matrix(y_test, y_pred, labels=None, sample_weight=None)

In [20]:
print(confus)

[[3983  139   10]
 [ 435  201   16]
 [ 152   36    5]]


In [22]:
accuracy = accuracy_score(y_test, y_pred)*100
print(accuracy)

84.16716897729556
